In [ ]:
from dotenv import load_dotenv
import os
import requests
from PIL import Image
import base64
import easyocr
import numpy as np

# .env 파일 로드
load_dotenv()

# 환경 변수에서 Hugging Face API 토큰 가져오기
api_token = os.getenv('HUGGING_FACE_API')

# EasyOCR 리더 초기화 (한글과 영어 지원)
ocr_reader = easyocr.Reader(['ko', 'en'])

# 이미지 파일을 읽어서 base64로 변환
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Hugging Face API 호출 함수
def query(image_path):
    API_URL = "https://api-inference.huggingface.co/models/Salesforce/blip-image-captioning-large"
    headers = {"Authorization": f"Bearer {api_token}"}  # Hugging Face API 토큰
    image_data = encode_image(image_path)

    # API 요청 데이터 (이미지를 base64로 변환 후 전송)
    payload = {
        "inputs": {
            "image": image_data
        }
    }

    # API 호출
    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API 호출 오류: {response.status_code}, {response.text}")

# 이미지 파일 경로
image_path = './testimage/test6.png'

# 1. EasyOCR을 사용하여 이미지 내 텍스트 추출
image = Image.open(image_path).convert('RGB')
ocr_result = ocr_reader.readtext(np.array(image))
ocr_text = ' '.join([res[1] for res in ocr_result])

# 2. Hugging Face API를 사용하여 이미지 캡션 생성
try:
    result = query(image_path)
    if isinstance(result, list) and len(result) > 0:  # 결과가 리스트 형식인지 확인
        generated_caption = result[0].get('generated_text', '캡션 생성 실패')
    else:
        generated_caption = '캡션 생성 실패: 결과 형식 오류'
except Exception as e:
    generated_caption = f"API 오류: {e}"

# 3. OCR 결과와 Hugging Face 캡션 결합
combined_caption = f"OCR Text: {ocr_text}\nGenerated Caption: {generated_caption}"

# 결과 출력
print(combined_caption)
